## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-20-23-49-22 +0000,nyt,Coast Guard Softens Stance on Hate and Hazing,https://www.nytimes.com/2025/11/20/us/politics...
1,2025-11-20-23-48-54 +0000,nyt,"Nydia Velázquez, a New York Trailblazer in Con...",https://www.nytimes.com/2025/11/20/nyregion/ny...
2,2025-11-20-23-48-12 +0000,nyt,Senate Democrats Investigate Kennedy Center De...,https://www.nytimes.com/2025/11/20/us/politics...
3,2025-11-20-23-47-44 +0000,nypost,Anna Kepner’s stepbrother was ‘obsessed’ with ...,https://nypost.com/2025/11/20/us-news/anna-kep...
4,2025-11-20-23-44-00 +0000,nypost,"UC Berkeley student, 19, dies after drowning i...",https://nypost.com/2025/11/20/us-news/uc-berke...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2370/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
22,trump,54
9,new,23
314,ukraine,16
98,house,15
214,epstein,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
365,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...,135
377,2025-11-20-01-41-44 +0000,wapo,U.S. pushes new Ukraine peace plan as Trump’s ...,https://www.washingtonpost.com/national-securi...,127
62,2025-11-20-22-15-38 +0000,cbc,Trump administration touts plan for new oil dr...,https://www.cbc.ca/news/world/trump-administra...,123
372,2025-11-20-02-14-00 +0000,wsj,Trump Administration Pushes New Plan for Endin...,https://www.wsj.com/world/trump-administration...,121
309,2025-11-20-09-59-52 +0000,cbc,Trump to meet NYC mayor-elect Mamdani at White...,https://www.cbc.ca/news/world/trump-to-meet-ny...,111


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
365,135,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...
354,90,2025-11-20-03-35-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
184,70,2025-11-20-17-20-17 +0000,nypost,Rubio casts doubt on pro-Russian Ukraine peac...,https://nypost.com/2025/11/20/world-news/rubio...
183,63,2025-11-20-17-20-47 +0000,nypost,Kamala Harris and Jill Biden come face-to-face...,https://nypost.com/2025/11/20/us-news/kamala-h...
342,41,2025-11-20-05-04-01 +0000,nypost,Miss Jamaica tumbles off stage and is carried ...,https://nypost.com/2025/11/20/entertainment/mi...
324,41,2025-11-20-08-35-04 +0000,wsj,Stock Market Today: Stock-Market Rebound Evapo...,https://www.wsj.com/livecoverage/jobs-report-b...
235,40,2025-11-20-14-07-21 +0000,nypost,Delayed government data show US added surprisi...,https://nypost.com/2025/11/20/business/delayed...
234,40,2025-11-20-14-18-22 +0000,nyt,Israeli Strikes Kill at Least 25 in Gaza Amid ...,https://www.nytimes.com/2025/11/19/world/middl...
71,35,2025-11-20-22-02-00 +0000,wsj,Gap Bumps Up Fiscal-Year Outlook as Third-Quar...,https://www.wsj.com/business/earnings/gap-bump...
84,34,2025-11-20-21-34-32 +0000,nyt,Trump Calls Democrats’ Military Video ‘Seditio...,https://www.nytimes.com/2025/11/20/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
